In [217]:
!pip install scikit-learn

In [218]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder


accident_df = pd.read_csv('../accident.csv')
#vehicle_df = pd.read_csv('../vehicle.csv')
filtered_vehicle_df = pd.read_csv('../filtered_vehicle.csv')
person_df = pd.read_csv('../person.csv')

# merge only on 'ACCIDENT_NO' will cause a huge duplication in data, that leads to 728,905 rows of data, which by checking
# 345,184 rows are actual duplication rows, that it  47.4%.

#merged_df = pd.merge(accident_df, filtered_vehicle_df, on='ACCIDENT_NO', how='inner')
#merged_df = pd.merge(merged_df, person_df, on='ACCIDENT_NO', how='inner')

# new merge method merges on both 'ACCIDENT_NO' and 'VEHICLE_ID'
vp = pd.merge(person_df, filtered_vehicle_df, on=['ACCIDENT_NO', 'VEHICLE_ID'], how='inner')
merged_df = pd.merge(vp, accident_df, on='ACCIDENT_NO', how='left')



Lets first look at the dimension of the data

In [ ]:
# show some examples of each feature
for col in merged_df.columns:
    print(f"{col}: {merged_df[col].dropna().unique()[:5]}")


ACCIDENT_NO: ['T20250000606' 'T20240012414' 'T20240001586' 'T20200004467'
 'T20190018772']
PERSON_ID: ['A' '01' 'B' '02' '04']
VEHICLE_ID: ['A' 'B' 'C' 'D' 'E']
SEX: ['M' 'F' 'U']
AGE_GROUP: ['40-49' '26-29' '18-21' '16-17' '22-25']
INJ_LEVEL: [3 4 2 1]
INJ_LEVEL_DESC: ['Other injury' 'Not injured' 'Serious injury' 'Fatality']
SEATING_POSITION: ['D' 'LF' 'OR' 'NK' 'RR']
HELMET_BELT_WORN: [9. 1. 8. 2. 3.]
ROAD_USER_TYPE: [2 3 9 7 1]
ROAD_USER_TYPE_DESC: ['Drivers' 'Passengers' 'Not Known' 'Pedestrians' 'Motorcyclists']
LICENCE_STATE: ['V' 'Z' 'O' 'W' 'D']
TAKEN_HOSPITAL: ['N' 'Y']
EJECTED_CODE: [0. 9. 1. 3. 2.]
VEHICLE_YEAR_MANUF: [2016. 2008. 2012. 2009. 1994.]
VEHICLE_DCA_CODE: [1. 2. 8. 3.]
INITIAL_DIRECTION: ['N' 'SW' 'E' 'S' 'NE']
ROAD_SURFACE_TYPE: [1. 9. 3. 2.]
ROAD_SURFACE_TYPE_DESC: ['Paved' 'Not known' 'Gravel' 'Unpaved']
REG_STATE: ['V' 'S' 'T' 'W' 'N']
VEHICLE_BODY_STYLE: ['SEDAN' 'WAGON' 'S WAG' 'UTIL' 'DC UTE']
VEHICLE_MAKE: ['TOYOTA' 'M MOVE' 'HOLDEN' 'HONDA' 'MAZDA']
VEH

In [ ]:
# number of features
merged_df.shape[1]

71

In [ ]:
# number of rows 
len(merged_df)

331993

In [ ]:
# check if there is any null value
with pd.option_context('display.max_rows', None):
    print(merged_df.isnull().sum().sort_values(ascending=False))


VEHICLE_POWER             331993
CUBIC_CAPACITY            302305
VEHICLE_WEIGHT            282216
CARRY_CAPACITY            282211
TAKEN_HOSPITAL            232911
LICENCE_STATE              75363
EJECTED_CODE               23523
RMA                        12720
VEHICLE_MODEL               2591
VEHICLE_DCA_CODE             706
CONSTRUCTION_TYPE            669
INITIAL_IMPACT               120
VEHICLE_BODY_STYLE            59
SEATING_POSITION              26
SEX                           22
TRAILER_TYPE                  16
VEHICLE_YEAR_MANUF             8
VEHICLE_COLOUR_2               2
HELMET_BELT_WORN               1
VEHICLE_MOVEMENT               1
PERSON_ID                      0
ACCIDENT_NO                    0
VEHICLE_ID                     0
INJ_LEVEL_DESC                 0
INJ_LEVEL                      0
AGE_GROUP                      0
ROAD_SURFACE_TYPE_DESC         0
VEHICLE_MAKE                   0
REG_STATE                      0
ROAD_SURFACE_TYPE              0
VEHICLE_TY

continue process the data, check if there is any uneccessary features

In [ ]:
# these features are considered to be useless with respect with training models
drop_feature = ['ACCIDENT_NO',
                'NODE_ID', 
                'PERSON_ID',
                'VEHICLE_ID',
                'ACCIDENT_TIME',
                'ACCIDENT_DATE',  # although date can potentially categorise into season or month, but the model dont need it in this case.
                'VEHICLE_MODEL',
                'VEHICLE_MAKE'
                ]
merged_df = merged_df.drop(columns=drop_feature)


In [ ]:
# lets see if theres some features that is categorical
with pd.option_context('display.max_rows', None):
    print(merged_df.nunique())

SEX                          3
AGE_GROUP                   14
INJ_LEVEL                    4
INJ_LEVEL_DESC               4
SEATING_POSITION             9
HELMET_BELT_WORN             9
ROAD_USER_TYPE               8
ROAD_USER_TYPE_DESC          6
LICENCE_STATE               11
TAKEN_HOSPITAL               2
EJECTED_CODE                 5
VEHICLE_YEAR_MANUF          64
VEHICLE_DCA_CODE             4
INITIAL_DIRECTION            9
ROAD_SURFACE_TYPE            4
ROAD_SURFACE_TYPE_DESC       4
REG_STATE                    7
VEHICLE_BODY_STYLE          87
VEHICLE_POWER                0
VEHICLE_TYPE                22
VEHICLE_TYPE_DESC           22
VEHICLE_WEIGHT            1208
CONSTRUCTION_TYPE            3
FUEL_TYPE                    8
NO_OF_WHEELS                10
NO_OF_CYLINDERS             24
SEATING_CAPACITY            61
TARE_WEIGHT               3676
TOTAL_NO_OCCUPANTS          40
CARRY_CAPACITY            3201
CUBIC_CAPACITY              99
FINAL_DIRECTION              9
DRIVER_I

it seems that there are some features that worth using one-hot to catagorise

first, there are some duplicated data that pairs with their description. to one hot these data, i will drop the desc features and keep others, since they represents the same thing. 

In [225]:
def drop_desc_keep_encoded(df):
    cols = df.columns
    cols_to_drop = []

    for col in cols:
        if col.endswith('_DESC'):
            prefix = col[:-5]
            if prefix in cols:
                cols_to_drop.append(col)

    cleaned_df = df.drop(columns=cols_to_drop)
    return cleaned_df, cols_to_drop



In [226]:
cleaned_df, dropped_desc_columns = drop_desc_keep_encoded(merged_df)
print("Deleted description columns:", dropped_desc_columns)

Deleted description columns: ['INJ_LEVEL_DESC', 'ROAD_USER_TYPE_DESC', 'ROAD_SURFACE_TYPE_DESC', 'VEHICLE_TYPE_DESC', 'TRAFFIC_CONTROL_DESC', 'ACCIDENT_TYPE_DESC', 'ROAD_GEOMETRY_DESC']


In [ ]:
# checking the new demsion of the cleaned df
for col in cleaned_df.columns:
    print(f"{col}: {cleaned_df[col].dropna().unique()[:5]}")


SEX: ['M' 'F' 'U']
AGE_GROUP: ['40-49' '26-29' '18-21' '16-17' '22-25']
INJ_LEVEL: [3 4 2 1]
SEATING_POSITION: ['D' 'LF' 'OR' 'NK' 'RR']
HELMET_BELT_WORN: [9. 1. 8. 2. 3.]
ROAD_USER_TYPE: [2 3 9 7 1]
LICENCE_STATE: ['V' 'Z' 'O' 'W' 'D']
TAKEN_HOSPITAL: ['N' 'Y']
EJECTED_CODE: [0. 9. 1. 3. 2.]
VEHICLE_YEAR_MANUF: [2016. 2008. 2012. 2009. 1994.]
VEHICLE_DCA_CODE: [1. 2. 8. 3.]
INITIAL_DIRECTION: ['N' 'SW' 'E' 'S' 'NE']
ROAD_SURFACE_TYPE: [1. 9. 3. 2.]
REG_STATE: ['V' 'S' 'T' 'W' 'N']
VEHICLE_BODY_STYLE: ['SEDAN' 'WAGON' 'S WAG' 'UTIL' 'DC UTE']
VEHICLE_POWER: []
VEHICLE_TYPE: [ 1  2  4 71 61]
VEHICLE_WEIGHT: [ 2100.  1805. 26000.  2805.  2800.]
CONSTRUCTION_TYPE: ['R' 'P' 'A']
FUEL_TYPE: ['M' 'P' 'D' 'G' 'E']
NO_OF_WHEELS: [ 4.  6.  8. 12. 10.]
NO_OF_CYLINDERS: [4. 6. 8. 5. 3.]
SEATING_CAPACITY: [ 5.  7.  2.  3. 42.]
TARE_WEIGHT: [1570. 1145. 1805. 1875. 1840.]
TOTAL_NO_OCCUPANTS: [1. 2. 5. 3. 4.]
CARRY_CAPACITY: [  530.   560. 15800.   985.  1305.]
CUBIC_CAPACITY: [1900. 2500. 3500. 400

In [228]:
cleaned_df.shape[1]

56

before one hot, i will keep a version of df for light GBM,

In [229]:
cleaned_df.to_csv('../merged_cleaned.csv', index=False)

In [ ]:
def one_hot_encode_expand(df, columns_to_encode, drop_first=True):
    df_encoded = pd.get_dummies(df, columns=columns_to_encode, drop_first=drop_first)
    return df_encoded

In [ ]:
def one_hot_encode_vectorise(df, columns):
    df_new = df.drop(columns=columns).copy()

    for col in columns:
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = encoder.fit_transform(df[[col]])
        # save as list 
        encoded_vectors = pd.Series(list(encoded), index=df.index)
        # name the new features as col + '_vec'
        df_new[col + '_vec'] = encoded_vectors
    return df_new


In [ ]:
one_hot_columns = [
    'ACCIDENT_TYPE_DESC',
    'DAY_WEEK_DESC',
    'LIGHT_CONDITION',
    'ROAD_GEOMETRY_DESC',
    'SPEED_ZONE',
    'RMA',
    'VEHICLE_TYPE_DESC',
    'REG_STATE',
    'FUEL_TYPE',
    'SEX',
    'AGE_GROUP',
    'INJ_LEVEL_DESC',
    'SEATING_POSITION',
    'HELMET_BELT_WORN',
    'ROAD_USER_TYPE_DESC',
    'LICENCE_STATE',
    'TRAFFIC_CONTROL_DESC',
    'INITIAL_DIRECTION',
    'FINAL_DIRECTION',
    'TRAILER_TYPE',
    'CONSTRUCTION_TYPE',
    'CAUGHT_FIRE',
    'TOWED_AWAY_FLAG',
    'TAKEN_HOSPITAL'
]


In [ ]:
# each one hot one hot category is encoded as a new feature, as result, there is 518 features
# merged_df_onehot_expand_df = one_hot_encode_expand(desc_cleaned_merged_df, one_hot_columns)

# vectorised one hot df, still 72 features, ready for neuron networking
merged_onehot_vectorized_df = one_hot_encode_vectorise(merged_df, one_hot_columns)


In [ ]:
merged_onehot_vectorized_df.head(1)

In [ ]:
merged_onehot_vectorized_df.to_csv('../merged_onehot_vectorized.csv', index=False)

If im going to use MLP models, then i will need to vectorise the continuous numeric data

In [ ]:
from pandas.api.types import is_numeric_dtype

# 找出数值列
numeric_cols = [col for col in merged_df.columns if is_numeric_dtype(merged_df[col])]

# 计算每列的最小值、最大值和范围
spread = merged_df[numeric_cols].agg(['min', 'max'])
spread.loc['range'] = spread.loc['max'] - spread.loc['min']

# 转置后查看范围最大的列（建议归一化）
spread.T.sort_values(by='range', ascending=False).head(20)


In [ ]:
merged_df_onehot_vectorized_df['SEVERITY'].value_counts(normalize=True)


In [ ]:
normalize_columns = [
    'NO_OF_VEHICLES',         
    'NO_PERSONS_KILLED',      
    'NO_PERSONS_INJ_2',       
    'NO_PERSONS_INJ_3',       
    'NO_PERSONS_NOT_INJ',     
    'NO_PERSONS',             
    'NO_OF_WHEELS',           
    'NO_OF_CYLINDERS',        
    'SEATING_CAPACITY',       
    'TARE_WEIGHT',            
    'TOTAL_NO_OCCUPANTS',     
    'CARRY_CAPACITY',         
    'CUBIC_CAPACITY',         
    'VEHICLE_WEIGHT'          
]